# Anomaly Detection System - Main Entry Point

This notebook serves as the main entry point for the anomaly detection system. It provides functions to:
- Train a new model
- Load and evaluate existing models
- Make predictions on new images
- Analyze model performance

## Import Required Libraries and Modules

In [ ]:
import os
import argparse
import logging
from datetime import datetime
import json
import sys
from solar_model import (
    train_and_evaluate,
    evaluate_existing_model,
    predict_single_image,
    load_best_model
)
import config as cfg

## Setup Logging Configuration

In [ ]:
def setup_logging(log_dir="logs"):
    """
    Configure logging to both file and console
    """
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = os.path.join(log_dir, f"anomaly_detection_{timestamp}.log")
    
    # Configure logging format
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )
    
    logging.info(f"Logging configured. Log file: {log_file}")

## Training Function

In [ ]:
def train_model():
    """
    Train a new model and save results
    """
    logging.info("Starting model training...")
    
    try:
        model, label_encoder, history = train_and_evaluate()
        
        # Save training history
        history_file = os.path.join(cfg.CHECKPOINT_DIR, 'training_history.json')
        history_dict = {
            'loss': [float(x) for x in history.history['loss']],
            'accuracy': [float(x) for x in history.history['accuracy']],
            'val_loss': [float(x) for x in history.history['val_loss']],
            'val_accuracy': [float(x) for x in history.history['val_accuracy']]
        }
        
        with open(history_file, 'w') as f:
            json.dump(history_dict, f, indent=4)
        
        logging.info(f"Training history saved to {history_file}")
        logging.info("Model training completed successfully")
        
        return True
        
    except Exception as e:
        logging.error(f"Error during model training: {str(e)}")
        return False

## Evaluation Function

In [ ]:
def evaluate_model(model_path=None):
    """
    Evaluate an existing model
    """
    logging.info("Starting model evaluation...")
    
    try:
        if model_path is None:
            model_path = cfg.MODEL_SAVE_PATH
        
        accuracy, report = evaluate_existing_model(model_path)
        
        # Save evaluation results
        results = {
            'accuracy': float(accuracy),
            'classification_report': report,
            'evaluation_timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        results_file = os.path.join(cfg.CHECKPOINT_DIR, 'evaluation_results.json')
        with open(results_file, 'w') as f:
            json.dump(results, f, indent=4)
        
        logging.info(f"Evaluation results saved to {results_file}")
        logging.info(f"Model accuracy: {accuracy:.4f}")
        
        return True
        
    except Exception as e:
        logging.error(f"Error during model evaluation: {str(e)}")
        return False

## Prediction Function

In [ ]:
def predict_image(image_path, model_path=None):
    """
    Make prediction on a single image
    """
    logging.info(f"Making prediction for image: {image_path}")
    
    try:
        if model_path is None:
            model_path = cfg.MODEL_SAVE_PATH
            
        predicted_class, confidence, class_scores = predict_single_image(
            image_path, 
            model_path
        )
        
        if predicted_class is not None:
            # Save prediction results
            results = {
                'image_path': image_path,
                'predicted_class': predicted_class,
                'confidence': float(confidence),
                'class_scores': {k: float(v) for k, v in class_scores.items()},
                'prediction_timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            
            # Create predictions directory if it doesn't exist
            pred_dir = os.path.join(cfg.CHECKPOINT_DIR, 'predictions')
            if not os.path.exists(pred_dir):
                os.makedirs(pred_dir)
            
            # Save prediction results
            filename = f"prediction_{os.path.basename(image_path)}.json"
            results_file = os.path.join(pred_dir, filename)
            
            with open(results_file, 'w') as f:
                json.dump(results, f, indent=4)
            
            logging.info(f"Prediction results saved to {results_file}")
            return True
            
        return False
        
    except Exception as e:
        logging.error(f"Error during prediction: {str(e)}")
        return False

## Command Line Interface

You can run this notebook as a script with the following command line arguments:
- `--train`: Train a new model
- `--evaluate`: Evaluate an existing model
- `--predict`: Make prediction on a single image
- `--model_path`: Path to the model file (optional)
- `--image_path`: Path to the image file (required for prediction)

In [ ]:
def parse_arguments():
    """
    Parse command line arguments
    """
    parser = argparse.ArgumentParser(description='Anomaly Detection System')
    
    parser.add_argument('--train', action='store_true',
                      help='Train a new model')
    
    parser.add_argument('--evaluate', action='store_true',
                      help='Evaluate an existing model')
    
    parser.add_argument('--predict', action='store_true',
                      help='Make prediction on a single image')
    
    parser.add_argument('--model_path', type=str,
                      help='Path to the model file')
    
    parser.add_argument('--image_path', type=str,
                      help='Path to the image file')
    
    return parser.parse_args()

## Main Function

In [ ]:
def main():
    """
    Main entry point for the application
    """
    # Set up logging
    setup_logging()
    
    # Parse command line arguments
    args = parse_arguments()
    
    if args.train:
        logging.info("Starting training process...")
        success = train_model()
        if success:
            logging.info("Training completed successfully")
        else:
            logging.error("Training failed")
            
    if args.evaluate:
        logging.info("Starting evaluation process...")
        success = evaluate_model(args.model_path)
        if success:
            logging.info("Evaluation completed successfully")
        else:
            logging.error("Evaluation failed")
            
    if args.predict:
        if args.image_path is None:
            logging.error("Image path is required for prediction")
            return
            
        logging.info("Starting prediction process...")
        success = predict_image(args.image_path, args.model_path)
        if success:
            logging.info("Prediction completed successfully")
        else:
            logging.error("Prediction failed")

## Example Usage

Here's how to use the functions defined above:

In [ ]:
# Train a new model
# main(['--train'])

# Evaluate an existing model
# main(['--evaluate', '--model_path', 'path/to/model.h5']) # can also be .pb

# Make a prediction
# main(['--predict', '--image_path', 'path/to/image.jpg'])